<a href="https://colab.research.google.com/github/kryuchkovdm/Distillation/blob/master/methods/Teacher_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def train_teacher(teacher,train_dataloader,optimizer):
  train_loss_set = []
  train_loss = 0
  teacher.to(device)
  # Переводим модель в training mode
  teacher.train()

  for step, batch in enumerate(tqdm(train_dataloader)):
    # Переводим данные на видеокарту
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
    
    # Обнуляем градиенты
      optimizer.zero_grad()
    
    # Прогоняем данные по слоям нейросети
      loss = teacher(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      train_loss_set.append(loss[0].item())  
    
    # Обратный прогон
      loss[0].backward()
      torch.nn.utils.clip_grad_norm_(teacher.parameters(), 1.0)
    # Шаг
      optimizer.step()

    # Обновляем loss
      train_loss += loss[0].item()
      # Empty cuda
      del b_input_ids
      del b_labels
      del b_input_mask
      torch.cuda.empty_cache()
  print(f'Лосс на обучении: {train_loss / len(train_dataloader)}')
  return train_loss / len(train_dataloader)

In [ ]:
def validate_teacher(validation_dataloader,teacher):
    teacher.eval()

    total = 0
    correct = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader):

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        

            loss = teacher(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            
        total_eval_loss += loss[0].item()
        del b_input_ids
        del b_labels
        del b_input_mask
        torch.cuda.empty_cache()
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    
    print("Лосс на тесте: {0:.2f}".format(avg_val_loss))
    return total_eval_loss / len(validation_dataloader)

In [ ]:
def train_loop_teacher(model, optim, train_loader, test_loader, n_epochs=5, sched=None):
    training_results = {"epoch": list(range(n_epochs)),
                        "train_loss": [],                   
                        "test_loss": []
                        }

    model.to(device)

    try:
        for i in range(n_epochs):
            
            train_loss = train_teacher(model,train_loader, optimizer)
            if sched is not None:
                sched.step()
            test_loss = validate_teacher(test_loader, model)
            training_results["train_loss"].append(train_loss)
            training_results["test_loss"].append(test_loss)
    except KeyboardInterrupt:
        pass

    return pd.DataFrame(training_results)